# Starting a binary black hole simulation

One of the systems that we are simulating with SpECTRE are binary black hole
(BBH) mergers and their gravitational waves at very high accuracy. In this
tutorial we will start a BBH simulation with SpECTRE and monitor its output.

First, make sure that you have compiled all the executables you need for running
BBH simulations. Make sure you do this on a compute node:

```sh
# Get a compute node
interact -f cascade -n 24 -m 190g -t 02:00:00
# Compile the code
cd $SPECTRE_BUILD
make -j24 cli bbh
```

This will compile:

- The command-line interface (CLI)
- SolveXcts (initial data)
- EvolveGhBinaryBlackHole (inspiral)
- EvolveGhSingleBlackHole (ringdown)

We will use the SpECTRE command-line interface (CLI) in this session. You have
learned how to access the CLI in the technology check session. Try it now by
exploring the `spectre bbh` subcommand of the CLI. You can run the commands in
this notebook or in your terminal:

In [10]:
%%bash
spectre bbh --help

Usage: spectre bbh [OPTIONS] COMMAND [ARGS]...

  Pipeline for binary black hole simulations.

Options:
  -h, --help  Show this message and exit.

Commands:
  find-horizon    Find an apparent horizon in volume data.
  generate-id     Generate initial data for a BBH simulation.
  postprocess-id  Postprocess initial data after generation.
  start-inspiral  Schedule an inspiral simulation from initial data.
  start-ringdown  Schedule a ringdown simulation from the inspiral.


These subcommands are part of the BBH pipeline, which you can use to start and
control BBH simulations. Steps in the BBH pipeline are, for example,

- Generating the initial data
- Running the inspiral simulation
- Running the ringdown simulation
- Extracting waveforms

> **Warning:** The BBH pipeline is still experimental and under active
> development. If you find any issues, let a core developer know.

All CLI commands are also available in Python, so everything you do with the CLI
you can also do in Python scripts. We will use Python to work with BBH data in
the next session.

## Generate initial data

Using the pipeline, you can generate initial data to start your BBH simulation
like this:

```sh
spectre bbh generate-id -q 1 --chi-A 0 0 0 --chi-B 0 0 0 -D 16 -w 0.015 -a 0 \
  --no-control -d ./runs/01_bbh
```

That's it! Now your job is scheduled on the cluster. Once the job runs, it will
generate initial data with the parameter you have specified. To get initial data
quickly we have disabled the control loop that drives the horizon masses and
spins to your specified parameters with `--no-control`. We could also add
`--evolve` to start the evolution of the initial data right after generation.
However, we'll start the inspiral in a separate command below to control the
parameters.

The initial data will be generated in the _pipeline directory_ (`-d`) that we
set to `./runs/01_bbh` in the command above. You will find a directory named
`001_InitialData` in the pipeline directory. The next step in the pipeline will
create `002_Inspiral`. If you're running just a single step of the pipeline, you
can also specify the output directory directly with `-o` (`-d` and `-o` are
mututally exclusive).

Of course you can change parameters such as the mass-ratio $q$, the black hole
spins $\mathbf{\chi}_A$ and $\mathbf{\chi}_B$, and the initial orbital
parameters:

- Coordinate separation $D_0$ (`-D`)
- Orbital angular velocity $\Omega_0$ (`-w`)
- Radial expansion velocity $\dot{a}_0$ (`-a`)

Alternatively, you can choose `--eccentricity=0` plus either `--num-orbits`, or
`--time-to-merger`, or one of the above orbital parameters to define the orbit
based on post-Newtonian estimates.

> To learn more about initial data generation you will have to wait until
> Friday.

The initial data generation should take about 5 minutes to complete at this
(fairly high) resolution on the 32 cores that we have available on Oscar.
However, because HPC systems can sometimes be unpredictable with how long jobs
will be in the queue, we've also run a full initial data solve already. You can
find this data at:

```
/oscar/data/icerm/knelli/workshop_materials/01_monday/pipeline_tutorial/001_InitialData
```

If your own initial data job doesn't finish by the time you get here, just
change the paths below to use this data.

In [13]:
%%bash
# Check if your job has finished
squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           4025979     batch EvolveGh     nvu8  R   17:02:17      1 node2301


## Starting the inspiral

The steps of the pipeline are connected, e.g. you can pass `--evolve` to
`spectre bbh generate-id` to automatically start the inspiral right after
initial data generation. However, you can also run each step in the pipeline on
its own, e.g. to start the inspiral with different parameters or in another
directory. Let's start the inspiral on just 1 node (with 32 cores), because
that's what we have available on Oscar:

```sh
spectre bbh start-inspiral runs/01_bbh/001_InitialData/InitialData.yaml \
  -N 1 -d ./runs/01_bbh
```

This will create `002_Inspiral` in your pipeline directory. In it, the first
_segment_ of your BBH simulation will start. A segment is a chunk of a
long-running simulation, which will terminate after some time (typically 1 day)
and resubmit itself so we can run on supercomputers with time-limited queues.

Note that you can also start the inspiral from SpEC data (if you happen to have
some):

```sh
spectre bbh start-inspiral /path/to/ID_Params.perl -O bbh_spec
```

While the BBH pipeline is still experimental, we recommend reviewing the
generated input files for any errors.

## Monitoring your simulation

Now you can monitor your simulation. Try running:

In [15]:
%%bash
# This just increases the column width so you can see everything in this
# notebook. You don't need this line if you're running this in a terminal.
export COLUMNS=150

spectre status

EvolveGhBinaryBlackHole ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

 State    End  JobID    JobName                  Elapsed   Cores  Nodes  Time [M]  Speed [M/h]  Orbits   Separation [M]  3-Index Constraint 
 RUNNING  -    4025979  EvolveGhBinaryBlackHole  17:04:22  32     1      204       10.7034      1.19422  6.70093         7.78e-05           


It will show you the state of your running simulations on this machine.

> **Pro tip:** Try `spectre status -w 5` in a separate terminal to keep watch of
> your simulations, updating every 5 seconds.

You can also run a dashboard to give you a visual summary of your running
simulations with lots of plots (this is still very experimental). Note that all
of the info in the dashboard can also be plotted with the CLI or in Python. The
dashboard just collects it all to give you an overview of your simulation. We'll
take a look at some of the things that are plotted in the next hands-on session.

```sh
python-spectre -m streamlit run $SPECTRE_HOME/tools/Status/Dashboard.py
```

We have the dashboard running on Oscar, so if you're feeling adventorous you can
forward port 8501 on Oscar's `vscode1` node and open the dashboard in your
browser:

- Dashboard URL: http://localhost:8501

### Example: Plot trajectories

You can use the spectre CLI to inspect the output data of the simulation while
it is running. We will explore simulation data in detail in the next tutorial.
For now, just try to plot the trajectories:

```sh
spectre plot trajectories runs/01_bbh/002_Inspiral/Segment_*/BbhReductions.h5 \
  -o trajectories.pdf
```

> Note: If you want to try this before your simulation has sufficiently
> progressed to show anything interesting, you can look at the complete
> simulation data in this directory:

In [18]:
%%bash
export BBH_DATA=/oscar/data/icerm/knelli/workshop_materials/01_monday/bbh_inspiral

spectre plot trajectories $BBH_DATA/BbhReductions.h5 -o runs/trajectories.pdf

![Trajectories](images/trajectories.png)

> **Pro tip:** You can also regularly regenerate plots like this to monitor your
> simulation. Try `watch spectre plot trajectories ...`.

### Measuring Eccentricity

We have basic support for measuring the orbital eccentricity and updating the
initial orbital parameters to reduce the eccentricity. Work on automation and
improvements of this procedure are ongoing, and mostly independent of the actual
simulation code. You can do a quick estimate of the orbital eccentricity like
this (we're using the sample data here because your simulation probably hasn't
progressed far enough yet):

In [17]:
%%bash
export BBH_DATA=/oscar/data/icerm/knelli/workshop_materials/01_monday/bbh_inspiral

spectre eccentricity-control $BBH_DATA/BbhReductions.h5 --tmin 500 --tmax 2000 -o runs/eccentricity.pdf

[13:13:16] INFO     Eccentricity control summary      ]8;id=349153;file:///users/nvu8/spectre/build/bin/python/spectre/Pipelines/EccentricityControl/EccentricityControl.py\EccentricityControl.py]8;;\:]8;id=247768;file:///users/nvu8/spectre/build/bin/python/spectre/Pipelines/EccentricityControl/EccentricityControl.py#244\244]8;;\
           INFO     ==== Function fitted to dD/dt:    ]8;id=159811;file:///users/nvu8/spectre/build/bin/python/spectre/Pipelines/EccentricityControl/EccentricityControl.py\EccentricityControl.py]8;;\:]8;id=393124;file:///users/nvu8/spectre/build/bin/python/spectre/Pipelines/EccentricityControl/EccentricityControl.py#250\250]8;;\
                    B*cos(w*t+np.pi/2)+const      ,                             
                    rms = 0.00341  ====                                         
           INFO     Fit parameters:                   ]8;id=353829;file:///users/nvu8/spectre/build/bin/python/spectre/Pipelines/EccentricityControl/Eccentrici

## Inspecting the input file

SpECTRE simulations are defined by an input file in YAML format. The input file
collects all options for the simulation and also specifies the C++ executable to
run it.

The inspiral input file that you just started running is here:

```
runs/01_bbh/002_Inspiral/Segment_0000/Inspiral.yaml
```

Open the input file to inspect it (either here in Jupyter, in a terminal with
vim/emacs, in VSCode, or with `cat /path/to/input/file`). We'll go through some
of the important options together.

## Reproducibility of simulations

A quick note on reproducibility, which is a crucial topic in scientific
computing:

SpECTRE simulations are reasonably well reproducible because (1) all options are
defined in the YAML input file, and (2) the input file and even the entire
source code used to compile the executable are stored in each H5 output file.

So, for the highest level of reproducibility you can extract the input file from
an H5 output file (see `spectre extract-input --help`), and also extract the
source code archive (see
[docs](https://spectre-code.org/tutorial_visualization.html#autotoc_md93)), then
compile and run the executable.

To help with reproducibility of results you publish in papers, we recommend that
you publish your input files as supplementary material and also cite the
specific code release with which you ran your simulations (see [Versioning and
releases](https://spectre-code.org/versioning_and_releases.html)).

## Troubleshooting a BBH simulation

If something fails (and of course it will), you can start by looking through the
data you have. First, look at the log file of your simulation. It's often named
`spectre.out` and located in the run directory of your simulation. Go to the
very end of the file and look for error messages. If you see many error
messages, the first one is often most helpful. Ideally, you'll find a stack
trace with the function calls that lead to the error. The error message and the
stack trace might already give you an idea as to what has happened.

It will also be helpful to analyse the output data that your simulation has
written up to this point, and look for anything fishy. In the next tutorial we
will go through a lot of ways to analyse output data from your simulations.

Once you have collected some clues as to what as happened, the best place to ask
for help is the `#spectre` channel on the SXS Slack or email the core developers
(core-devs@spectre-code.org). If your run is on a shared supercomputer then send
along a file system path to your run and make sure it is publicly readable.

### Incoming characteristic speeds error

By far the most common error you'll see in BBH runs with SpECTRE (and with SpEC for that matter) is something we refer to as "incoming char speeds". The error will look something like this:

```
DemandOutgoingCharSpeeds boundary condition violated with speed index 3 ingoing: -7.02605e-05
 speed: <a lot of numbers here that don't really matter>

See gh::characteristic_speeds for the index ordering of characteristic speeds

In element:[B14,(L1I1,L1I0,L2I0)]
In direction: -2
############ ERROR ############

Type: SpectreError
```

While very common to get, this error is only a symptom of what really went
wrong. Basically what's happening here is that with the Generalized Harmonic
formulation, we have characteristic fields of the first order evolution
equations that read as

$$
\begin{align}
  w^{g}_{ab} &= g_{ab}, \\
  w^{0}_{iab} &= (\delta^k_i-s^k s_i)\Phi_{kab}, \\
  w^{\pm}_{ab} &= \Pi_{ab}\pm s^i\Phi_{iab} -\gamma_2 g_{ab},
\end{align}
$$
with associated characteristic speeds
$$
\begin{align}
  \lambda_{w^g} =& -(1+\gamma_1)\beta^i s_i -v^i_g s_i, \\
  \lambda_{w^0} =& -\beta^i s_i -v^i_g s_i, \\
  \lambda_{w^\pm} =& \pm \alpha - \beta^i s_i - v^i_g s_i,
\end{align}
$$

Specifically, the incoming char speed error is that $\lambda_{w^-} < 0$ which
means that we have information flowing into the computational domain from the
excision surfaces. The issue is that we don't impose boundary conditions on the
excisions. (think, what would those boundary conditions actually be?) Therefore,
when $\lambda_{w^-} < 0$, we just error out because there's nothing else to do.

In order to find the real cause of what went wrong, you have to analyze all the
output from the simulation. The first thing to look at in this case is what the
control system is doing (which controls the size and shape of the excision
surfaces). There are two CLI commands that will give a good overview of what
happened (we use the sample BBH data here again):

In [16]:
%%bash
export BBH_DATA=/oscar/data/icerm/knelli/workshop_materials/01_monday/bbh_inspiral

# There are two BHs, A and B. Choose A for now
spectre plot size-control -d A -o runs/SizeA.pdf $BBH_DATA/BbhReductions.h5
spectre plot control-system -o runs/ControlSystem.pdf $BBH_DATA/BbhReductions.h5

Take a look at the PDFs that were produced. We won't go into details about these
plots, but if you include these in your message to the Slack or core devs,
they'll be extremely grateful :)

## Running other simulations

We have so far only built a pipeline for BBH simulations. Other executables can
be run by editing a YAML input file and passing it to the executable. Many
executables have example input files in the `tests/InputFiles/` directory that
you can take as a starting point. For example, we can run a quick scalar wave
like this:

```sh
spectre run $SPECTRE_HOME/tests/InputFiles/ScalarWave/PlaneWave3D.yaml -o runs/02_scalar_wave
```

The `run` command (or its alias `schedule`) will submit a job with the input
file. It also allows you to put placeholders in the input file that can be
filled from the command line. For example, make this change in
`$SPECTRE_HOME/tests/InputFiles/PlaneWave3D.yaml`:

```diff
@@ -20,7 +20,7 @@ InitialData:
     Center: [0.0, 0.0, 0.0]
     Profile:
       Sinusoid:
-        Amplitude: 1.0
+        Amplitude: {{ amplitude }}
         Wavenumber: 1.0
         Phase: 0.0
```

Now you can run it like this:

```sh
spectre run $SPECTRE_HOME/tests/InputFiles/PlaneWave3D.yaml \
  -o "test_scalar_wave_A{{amplitude}}" -p amplitude=1,2,3
```

Three jobs will be run, each with a different value of the amplitude.

You can also run executables manually, like this:

```sh
$SPECTRE_BUILD/EvolveScalarWave3D --input-file INPUT_FILE
```

With this method you have to deal with output directories, submit scripts, etc.
yourself, so we recommend using `spectre run`/`spectre schedule` instead.